# TMS LVDT sensing matrix construction

<br>
<div align="center">K. Okutomi<br>
2018.08.24
<br></div>

Check the construction procedure of the sensing matrix for the TMS LVDTs.

In [28]:
import numpy as np

pi = np.pi
cos = lambda x: np.cos(x/180*pi)    # cos in [deg]
sin = lambda x: np.sin(x/180*pi)    # sin in [deg]

##  LVDT geometry information (copied from the excel file)
#
#              radius  azimuth  height 
#                [mm]    [deg]    [mm]
#       -------------------------------
LVDT = {'H1': [497.21, -148.24,   0.01], 
        'H2': [497.21,  -28.24,   0.01], 
        'H3': [497.21,   91.76,   0.01],
        'V1': [504.56,  159.93, -78.06], 
        'V2': [504.56,  -80.07, -78.06], 
        'V3': [504.56,   39.93, -78.06]}

## Abstract

Denoting the radius in the $L$-$T$ plane as $(r_h,\,r_v)$ for horitontal and vertical LVDTs, azimuth as $\theta_i$ ($i = \mathrm{\{H1,\,H2,\,H3,\,V1,\,V2,\,V3\}}$), and the depth (negative height) as $d_v$ (only the depths for the vertical LVDTs taken into account), we can write the simultaneous equations in a matrix form like the following.

\begin{align}
\left[\begin{matrix}
\mathrm{V1} \\ \mathrm{V2} \\ \mathrm{V3} \\ \mathrm{H1} \\ \mathrm{H2} \\ \mathrm{H3} 
\end{matrix}\right] =
\boldsymbol{M}\;
\left[\begin{matrix}
L \\ T \\ V \\ R \\ P \\ Y
\end{matrix}\right]\;,
\end{align}

where,

\begin{align}
\boldsymbol{M} =
\left[\begin{matrix}
0 & 0 & 1 & r_\mathrm{V1}\cos(\arctan\frac{d_v}{r\sin\theta_\mathrm{V1}}) & r_\mathrm{V1}\cos(\arctan\frac{d_v}{r\cos\theta_\mathrm{V1}}) & 0 \\
0 & 0 & 1 & r_\mathrm{V2}\cos(\arctan\frac{d_v}{r\sin\theta_\mathrm{V2}}) & r_\mathrm{V2}\cos(\arctan\frac{d_v}{r\cos\theta_\mathrm{V2}}) & 0 \\
0 & 0 & 1 & r_\mathrm{V3}\cos(\arctan\frac{d_v}{r\sin\theta_\mathrm{V3}}) & r_\mathrm{V3}\cos(\arctan\frac{d_v}{r\cos\theta_\mathrm{V3}}) & 0 \\
\cos\phi_\mathrm{H1} & \sin\phi_\mathrm{H1} & 0 & 0 & 0 & r_h \\  
\cos\phi_\mathrm{H2} & \sin\phi_\mathrm{H2} & 0 & 0 & 0 & r_h \\  
\cos\phi_\mathrm{H3} & \sin\phi_\mathrm{H3} & 0 & 0 & 0 & r_h \\  
\end{matrix}\right]\;,
\end{align}

and

\begin{align}
\phi_i &= \theta_i + \pi/2 \;,\\
r_i &= \sqrt{r_v^2\sin^2\theta_i + d_v^2}\;.
\end{align}

## Memo

* A cartoon of sensor configuration of the TMS LVDT
<img src="https://github.com/etoal83/images/blob/master/cartoons/ctn_TMSLVDTconfig.jpg?raw=true" width="600px"></img>

### Horizontal
* Each sensor basis (unit vector aligned to the sensitive direction) can be regarded as its radial vector rotated by 90$^\circ$.
* Given the azimuth information, it's easy to calculate the contribution of each DoF to one sensor.

### Vertical
* Comparing to the horizontal motion, the vertical sensors have relatively complicated response to the tilt rotation
* 

In [21]:
def hcoeff(key, dof):
    r = LVDT[key][0] * 1e-3   # [m]
    phi = LVDT[key][1] + 90   # [deg]
    dofcalc = {'L': cos(phi),
               'T': sin(phi),
               'Y': r}
    return dofcalc[dof]
    # For horizontal LVDTs, the height are ignored because they're small enough

    
def vcoeff(key, dof):
    r = LVDT[key][0] * 1e-3         # [m]
    theta = LVDT[key][1]            # [deg]
    depth = -LVDT[key][2] * 1e-3    # [m]
    rv = np.sqrt(r**2 * sin(theta)**2 + depth**2)
    dofcalc = {'V': 1,
               'R': rv * cos(np.arctan2(depth, r*sin(theta))*180/pi),
               'P': rv * cos(np.arctan2(depth, r*cos(theta))*180/pi)}
    return dofcalc[dof]    

In [33]:
# 3x3 horizontal submatrix
M_h = np.zeros([3, 3])
for i, key in enumerate(['H1', 'H2', 'H3']):
    for j, dof in enumerate(['L', 'T', 'Y']):
        M_h[i, j] = hcoeff(key, dof)

# 3x3 vertical submatrix
M_v = np.zeros([3, 3])
for i, key in enumerate(['V1', 'V2', 'V3']):
    for j, dof in enumerate(['V', 'R', 'P']):
        M_v[i, j] = vcoeff(key, dof)

# 6x6 full matrix of "M"
M = np.zeros([6, 6])
M[0:3, 2:5] = M_v
M[3:, 0:2]  = M_h[:, 0:2]
M[3:, 5]    = M_h[:, 2]

print('\nM = \n\n{0}\n'.format(M))


M = 

[[ 0.          0.          1.          0.17314882 -0.18740611  0.        ]
 [ 0.          0.          1.         -0.49700127  0.37447694  0.        ]
 [ 0.          0.          1.          0.32385246  0.3265477   0.        ]
 [ 0.52636233 -0.85026037  0.          0.          0.          0.49721   ]
 [ 0.47316592  0.88097333  0.          0.          0.          0.49721   ]
 [-0.99952825 -0.03071296  0.          0.          0.          0.49721   ]]



In [35]:
##  Final result of the sensing matrix LVDT2EUL
sensmat = np.linalg.inv(M)
print('\nLVDT2EUL = \n\n{0}\n'.format(sensmat))


LVDT2EUL = 

[[-0.         -0.         -0.          0.35090822  0.31544394 -0.66635216]
 [ 0.          0.          0.         -0.56684025  0.58731556 -0.02047531]
 [ 0.66084188  0.2732048   0.06595332  0.          0.          0.        ]
 [-0.11169609 -1.19773713  1.30943323 -0.         -0.         -0.        ]
 [-1.91294817  0.35120539  1.56174278 -0.         -0.         -0.        ]
 [ 0.          0.          0.          0.67040754  0.67040754  0.67040754]]



## Contact

* koki.okutomi(´・ω・｀)nao.ac.jp

Please change the _(´・ω・｀)_ to @ (at_mark)